In [0]:
from google.colab import files 
uploaded = files.upload() 

Saving data.csv to data.csv


In [2]:
import io 
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['data.csv'])) 
print(type(data))
print(data.head)

<class 'pandas.core.frame.DataFrame'>
<bound method NDFrame.head of                                                    URLs  ... Label
0     http://www.bbc.com/news/world-us-canada-414191...  ...     1
1     https://www.reuters.com/article/us-filmfestiva...  ...     1
2     https://www.nytimes.com/2017/10/09/us/politics...  ...     1
3     https://www.reuters.com/article/us-mexico-oil-...  ...     1
4     http://www.cnn.com/videos/cnnmoney/2017/10/08/...  ...     1
...                                                 ...  ...   ...
4004  http://beforeitsnews.com/sports/2017/09/trends...  ...     0
4005  http://beforeitsnews.com/u-s-politics/2017/10/...  ...     0
4006  https://www.activistpost.com/2017/09/ron-paul-...  ...     0
4007  https://www.reuters.com/article/us-china-pharm...  ...     1
4008  http://beforeitsnews.com/u-s-politics/2017/10/...  ...     0

[4009 rows x 4 columns]>


In [3]:
url_data=data.dropna()
url_data.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [4]:
url_data.shape

(3988, 4)

In [5]:
data_x=url_data.iloc[:,:-1]
data_x.head()

,URLs,Headline,Body
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed..."
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin..."


In [6]:
data_y=url_data.iloc[:,3:]
data_y.head()

,Label
0,1
1,1
2,1
3,1
4,1


In [7]:
print(data_x.shape)
print(data_y.shape)

(3988, 3)
(3988, 1)


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

In [0]:
def split_data(x,y):
    
    """
    Reminder : Clean the set before sending here.
    Input  : x=Attributes of the dataset, y=Target Attribute
    Output : Split data of train test
    
    We are removing dev , as dev can be better used for the models that includes DNN, CNN and RNN
    
    """
    X=x
    Y=y
    
    train_x,test_x,train_y,test_y = train_test_split(x,y,test_size=0.2,random_state=12)
    
    #train_x : Training data with attributes
    #train_y : Training data with ratings
    #test_x  : testing data with attributes
    #test_y  : testing data with rating 
    
    print("Final size of train/test :",train_x.size,"/",test_x.size)
    return train_x,train_y,test_x,test_y

In [0]:
def passive_aggressive(train_x,train_y,test_x,test_y,stop=False):
    
    if stop==True:
        print("Early Stopping is being used")
    
    pac= PassiveAggressiveClassifier(early_stopping=stop)
    pac.fit(train_x, train_y)
    predicted_pac_train=pac.predict(train_x)
    predicted_pac_test= pac.predict(test_x)
    
    score_train= metrics.accuracy_score(train_y, predicted_pac_train)
    score_test=metrics.accuracy_score(test_y,predicted_pac_test)
    
    print("train accuracy:",score_train)
    cm_pac_train= metrics.confusion_matrix(train_y, predicted_pac_train)
    print("train confusion matrix")
    print(cm_pac_train)
    print("test accuracy:",score_test)
    cm_pac_test= metrics.confusion_matrix(test_y, predicted_pac_test)
    print("test confusion matrix")
    print(cm_pac_test)

In [0]:
def mult_tfidf_pac(x1,x2,y,x3,stop=False):
    from sklearn.feature_extraction.text import TfidfVectorizer

    
    train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
    train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
    train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
    tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7)   # a TFIDF vectorizer  
    tfidf_train_x1 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x1).todense())  #fitting the training data
    tfidf_test_x1 = pd.DataFrame(tfidf_vectorizer.transform(test_x1).todense())

    tfidf_train_x2 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x2).todense())  #fitting the training data
    tfidf_test_x2 = pd.DataFrame(tfidf_vectorizer.transform(test_x2).todense()) 
        
    tfidf_train_x3 = pd.DataFrame(tfidf_vectorizer.fit_transform(train_x3).todense())  #fitting the training data
    tfidf_test_x3 = pd.DataFrame(tfidf_vectorizer.transform(test_x3).todense())
        
    train=pd.concat([tfidf_train_x1,tfidf_train_x2,tfidf_train_x3],axis=1)
    test=pd.concat([tfidf_test_x1,tfidf_test_x2,tfidf_test_x3],axis=1)
        
    passive_aggressive(train,train_y2,test,test_y2,stop)

In [36]:
mult_tfidf_pac(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body,stop=True) #early stopping true

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Early Stopping is being used
train accuracy: 1.0
train confusion matrix
[[1688    0]
 [   0 1502]]
test accuracy: 0.9987468671679198
test confusion matrix
[[432   0]
 [  1 365]]


In [39]:
mult_tfidf_pac(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body) #early stopping false

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
train accuracy: 1.0
train confusion matrix
[[1688    0]
 [   0 1502]]
test accuracy: 0.9987468671679198
test confusion matrix
[[432   0]
 [  1 365]]


In [0]:
def multi_hashv_pac(x1,x2,y,x3,stop=False):
    from sklearn.feature_extraction.text import HashingVectorizer
    train_x1,train_y1,test_x1,test_y1=split_data(x1,y)
    train_x2,train_y2,test_x2,test_y2=split_data(x2,y)
    train_x3,train_y3,test_x3,test_y3=split_data(x3,y)
        
    hash_vectorizer = HashingVectorizer(stop_words='english',n_features = 2**8)   # a HASH vectorizer , changed n_features
    hash_train_x1 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x1).todense()) ) #fitting the training data
    hash_test_x1 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x1).todense()))

    hash_train_x2 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x2).todense()) ) #fitting the training data
    hash_test_x2 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x2).todense()) )
        
    hash_train_x3 = pd.DataFrame(np.absolute(hash_vectorizer.fit_transform(train_x3).todense()))  #fitting the training data
    hash_test_x3 = pd.DataFrame(np.absolute(hash_vectorizer.transform(test_x3).todense()))
        
    train=pd.concat([hash_train_x1,hash_train_x2,hash_train_x3],axis=1)
    test=pd.concat([hash_test_x1,hash_test_x2,hash_test_x3],axis=1)
        
    passive_aggressive(train,train_y2,test,test_y2,stop)

In [38]:
multi_hashv_pac(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body) #early stopping false

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
train accuracy: 1.0
train confusion matrix
[[1688    0]
 [   0 1502]]
test accuracy: 0.9974937343358395
test confusion matrix
[[431   1]
 [  1 365]]


In [40]:
multi_hashv_pac(x1=data_x.URLs,x2=data_x.Headline,y=data_y.Label,x3=data_x.Body,stop=True) #early stopping true

Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Final size of train/test : 3190 / 798
Early Stopping is being used
train accuracy: 0.9993730407523511
train confusion matrix
[[1686    2]
 [   0 1502]]
test accuracy: 0.9962406015037594
test confusion matrix
[[430   2]
 [  1 365]]
